## 転移学習と微調整
まず、保存されているモデルを読み込みます。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
model=tf.keras.models.load_model('/content/drive/MyDrive/my_mnist_model.h5')

データを読み込み、正規化します。

In [ ]:
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images  = training_images / 255.0
test_images = test_images / 255.0

最終的な検証用の損失と精度を数値で確認しておきます。

In [ ]:
model.evaluate(test_images, test_labels)

## 別のデータセットへの転用
別のデータセットを読み込みます。データ件数が少ない場合を試したいので、それぞれ件数を1/10に減らして訓練してみましょう。

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = fashion_mnist.load_data()

(train_size, test_size) = (len(training_images), len(test_images))
(training_images, training_labels) = (training_images[:train_size//10], training_labels[:train_size//10])
(test_images, test_labels) = (test_images[:test_size//10], test_labels[:test_size//10])

training_images  = training_images / 255.0
test_images = test_images / 255.0
print(len(training_images), len(test_images))

新しいデータセットには元のモデルは全く性能が出ないことを確認しておきます。

In [ ]:
model.evaluate(test_images, test_labels)

転移学習を行います。最終層以外を凍結します。

In [ ]:
for layer in model.layers[:-1]:
    layer.trainable=False
model.layers[-1].trainable=True

モデルを確認すると、Trainable paramsが少ないことから、多くの層が凍結されていることが確認できます。

In [ ]:
model.summary()

コンパイルし、転移学習してみます。1階層しか重みを変更しないため、1エポックあたりの計算時間は短縮されているはずです。

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
train_hist=model.fit(training_images, training_labels, epochs=20, verbose=1,
                     validation_data=(test_images, test_labels), batch_size=128)

データ件数が少ないものの、それなりに収束はしますが、性能がそれほど良くないことが確認できます。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
figs, axs=plt.subplots(1, 2, figsize=(12, 4))
axs[0].plot(train_hist.history['loss'], label='training')
axs[0].plot(train_hist.history['val_loss'], label='validation')
axs[0].legend()
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[1].plot(train_hist.history['accuracy'], label='training')
axs[1].plot(train_hist.history['val_accuracy'], label='validation')
axs[1].legend()
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
plt.show()

さらに微調整してみます。ここでは全層を解凍し、学習率を1e-5に設定して20エポックほど訓練してみましょう。

In [ ]:
for layer in model.layers:
    layer.trainable=True
model.compile(optimizer = tf.optimizers.Adam(1e-4),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
train_hist=model.fit(training_images, training_labels, epochs=20, verbose=1,
                     validation_data=(test_images, test_labels), batch_size=128)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
figs, axs=plt.subplots(1, 2, figsize=(12, 4))
axs[0].plot(train_hist.history['loss'], label='training')
axs[0].plot(train_hist.history['val_loss'], label='validation')
axs[0].legend()
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[1].plot(train_hist.history['accuracy'], label='training')
axs[1].plot(train_hist.history['val_accuracy'], label='validation')
axs[1].legend()
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
plt.show()

少ないデータ件数、少ない計算時間で、それなりの性能が出るまでに訓練できることを確認できました。